## Load Bert Model

In [1]:
# BERT Selector

import numpy as np
import torch
from transformers import BertTokenizer

# Specify your cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def load_model(model_path, tokenizer_path):
    """
    Load the pre-trained model and tokenizer from the specified paths.

    Args:
    - model_path (str): Path to the saved model.
    - tokenizer_path (str): Path to the tokenizer.

    Returns:
    - model: The pre-trained model.
    - tokenizer: The pre-trained tokenizer.
    """
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)
    model = torch.load(model_path)
    model.to(device)
    model.eval()
    print("Model loaded and set to evaluation mode.")
    return model, tokenizer


def evaluate_email(input_text, model, tokenizer, max_padding=512):

    input_ids = []
    attention_masks = []
    texts = [input_text]
    for text in texts:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
            text,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=max_padding,  # Pad & truncate all sentences.
            pad_to_max_length=True,
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors="pt",  # Return pytorch tensors.
        )

        input_ids.append(encoded_dict["input_ids"])
        attention_masks.append(encoded_dict["attention_mask"])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor([[0, 1, 2, 3, 4, 5]]).to(torch.int64)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Model loaded and set to evaluation mode.")
    model.eval()

    # Evaluate data for one epoch
    b_input_ids = input_ids.to(device)
    b_input_mask = attention_masks.to(device)
    b_labels = labels.to(device)
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = output.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()

    probabilities = np.exp(
        logits - np.max(logits, axis=1, keepdims=True)
    )  # Stabilized softmax
    probabilities = probabilities / np.sum(probabilities, axis=1, keepdims=True)

    # Get the mapping of categories to codes before converting to codes
    ability_mapping = {0: "CR", 1: "CT", 2: "ICL", 3: "IF", 4: "MT", 5: "SUMM"}

    print(f"pred_flat : {probabilities}")
    print(f"labels_flat : {label_ids}")
    print(f"mapping: {ability_mapping}")

    output = ""
    prob_dict = {}
    for i, prob in enumerate(probabilities):
        for code, category in ability_mapping.items():
            output += f"{category}: {prob[code]:.2f} "
            prob_dict[category] = prob[code]

    print(output)
    return output, prob_dict


def find_optimal_temperature(
    prompt, model_name, grouped_avg_accuracy, model, tokenizer
):

    performance_map = grouped_avg_accuracy[
        grouped_avg_accuracy["model_name"] == model_name
    ]

    best_performance_prediction = -float("inf")
    best_temperature = None
    _, prediction_dict = evaluate_email(prompt, model, tokenizer, max_padding=512)
    predicted_category = max(prediction_dict, key=prediction_dict.get)

    for temperature in np.arange(0.1, 2.0, 0.3):
        temperature = round(temperature, 2)
        matching_rows = performance_map[
            (performance_map["temperature"] == temperature)
            & (performance_map["category"] == predicted_category)
        ]

        if not matching_rows.empty:
            performance_prediction = matching_rows["accuracy"].values[0]

            if performance_prediction > best_performance_prediction:
                best_performance_prediction = performance_prediction
                best_temperature = temperature

    return best_temperature, best_performance_prediction


# model_path = "/home/snt/projects_lujun/temperature_eval/bert_model_target_2"
# tokenizer_path = "/home/snt/llm_models/bert-base-multilingual-uncased"
# tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)
# model = torch.load(
#     "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/training/bert_model_target_8"
# )
# email = """Prompt:

# Please summarize the following content, ensuring that key points are covered and expressed concisely:

# Given Content:

# In recent years, the impacts of global climate change have become increasingly evident, especially in the form of frequent extreme weather events. Scientists have found that the rise in greenhouse gas emissions, particularly carbon dioxide and methane, is accelerating global temperature increases. This warming trend has led to the intensification of extreme weather phenomena, including heatwaves, floods, and powerful hurricanes. In response to this challenge, governments and environmental organizations worldwide are actively promoting emission reduction policies and the development of green energy. However, despite global efforts, the rate of increase in greenhouse gas emissions has not significantly slowed, and the pace of global warming has not been effectively controlled. As a result, scientists are calling on countries to intensify their actions and implement stricter climate policies to mitigate climate change and avoid potentially catastrophic consequences in the future.
# """

# output, prob_dict = evaluate_email(email, model, tokenizer, max_padding=512)

/home/snt/miniconda3/envs/causal_env_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Datasets

In [2]:
import os
import re
import pandas as pd
import json

folder_path = "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Results_General/evaluation"

dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith(".jsonl"):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, "r") as file:
            records = [json.loads(line) for line in file if line.strip()]
            df = pd.DataFrame(records)
            if "Qwen2.5-1.5B-Instruct" in filename:
                model_name = "Qwen2.5-1.5B-Instruct"
            elif "Phi-3.5-mini-instruct" in filename:
                model_name = "Phi-3.5-mini-instruct"
            elif "Llama-3.2-3B-Instruct" in filename:
                model_name = "Llama-3.2-3B-Instruct"
            elif "Qwen2.5-3B-Instruct" in filename:
                model_name = "Qwen2.5-3B-Instruct"
            elif "Llama-3.2-1B-Instruct" in filename:
                model_name = "Llama-3.2-1B-Instruct"
            elif "Llama-2-7b-chat-hf" in filename:
                model_name = "Llama-2-7b-chat-hf"
            elif "Llama-2-13b-chat-hf" in filename:
                model_name = "Llama-2-13b-chat-hf"
            elif "Llama-2-70b-chat-hf" in filename:
                model_name = "Llama-2-70b-chat-hf"
            elif "Meta-Llama-3-8B-Instruct" in filename:
                model_name = "Meta-Llama-3-8B-Instruct"
            elif "Meta-Llama-3-70B-Instruct" in filename:
                model_name = "Meta-Llama-3-70B-Instruct"
            elif "Mistral-7B-Instruct-v0.2" in filename:
                model_name = "Mistral-7B-Instruct-v0.2"
            elif "Mixtral-8x7B-Instruct-v0.1" in filename:
                model_name = "Mixtral-8x7B-Instruct-v0.1"
            else:
                model_name = "Unknown"
            df["model_name"] = model_name
            if "accuracy" in df.columns:
                df["accuracy"] = df["accuracy"].astype(float)
            else:
                df["accuracy"] = df.apply(
                    lambda row: (
                        row[f"{row['category']}_accuracy"]
                        if f"{row['category']}_accuracy" in df.columns
                        else None
                    ),
                    axis=1,
                )
            df["temperature"] = pd.to_numeric(df["temperature"], errors="coerce")
            df["temperature"] = df["temperature"].apply(
                lambda x: round(x, 2) if pd.notna(x) else x
            )
            columns_to_keep = ["model_name", "temperature", "category", "accuracy"]
            df = df[columns_to_keep]
            dfs.append(df)

In [3]:
import numpy as np

df = pd.concat(dfs, ignore_index=True)
df["temperature"] = df["temperature"].astype(float).round(2)

grouped_avg_accuracy = (
    df.groupby(["model_name", "temperature", "category"])["accuracy"]
    .mean()
    .reset_index()
)

grouped_avg_accuracy["temperature"] = grouped_avg_accuracy["temperature"].round(2)

tokenizer_path = "/home/snt/llm_models/bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)
model = torch.load(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/training/bert_model_target_8"
)

prompt = """
Prompt:

The year is 2075. Decades of unchecked climate change have transformed Earth into an unpredictable, chaotic environment. Cities lie submerged, deserts stretch endlessly, and hurricanes form with unnatural precision. Despite the warning signs, humanity's response came too late—or did it?

Write a short story from the perspective of a climate scientist who stumbles upon an abandoned research facility containing groundbreaking technology. This discovery could reverse the effects of climate change, but using it carries immense risks. As the protagonist grapples with this ethical dilemma, explore their internal conflict, the societal implications of their choice, and how the scars of a warming planet have shaped humanity's resilience.
"""


optimal_temperature, _ = find_optimal_temperature(
    prompt=prompt,
    model_name="Llama-2-7b-chat-hf",
    grouped_avg_accuracy=grouped_avg_accuracy,
    model=model,
    tokenizer=tokenizer,
)

print(optimal_temperature)

/tmp/ipykernel_382701/2080009334.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(
Truncation was not explicitly activated but `max_length` is provide

Model loaded and set to evaluation mode.
pred_flat : [[0.03827352 0.78679097 0.02313212 0.04687883 0.03759086 0.0673337 ]]
labels_flat : [[0 1 2 3 4 5]]
mapping: {0: 'CR', 1: 'CT', 2: 'ICL', 3: 'IF', 4: 'MT', 5: 'SUMM'}
CR: 0.04 CT: 0.79 ICL: 0.02 IF: 0.05 MT: 0.04 SUMM: 0.07 
1.3


In [4]:
from datasets import load_dataset
import requests
import os
from datasets import load_from_disk

tasks = ["boolq", "cb", "copa", "multirc", "record", "rte", "wic", "wsc"]
save_path = (
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets"
)


datasets = {}
for task in tasks:
    dataset_path = os.path.join(save_path, task)
    datasets[task] = load_from_disk(dataset_path)
    print(f"Dataset {task} loaded from {dataset_path}")


server_url = "http://0.0.0.0:8000/v1/chat/completions"
dataset_label = "validation"

output_name = f"Llama-3.2-1B-Instruct_{dataset_label}"

for task, dataset in datasets.items():
    print(f"Task: {task}")

    # Show the structure of the dataset for the current task
    print(dataset)
    print(dataset["train"][0])
    if "label" in dataset["validation"][0]:  # Check if 'label' key exists
        # Get all unique labels from the test set
        labels = set(example["label"] for example in dataset["validation"])
        print(f"Unique labels in the test set: {labels}")
    else:
        print("No 'label' field in the test set.")

options = {  # Configuration options for model generation
    "temperature": 1.0,
    "max_tokens": 1024,
    "top_p": 0.9,
    "frequency_penalty": 0.0,
    "seed": 47,  # Default
    "n": 1,
}


def generate_text_with_vllm(
    prompt,
    model_name="/home/snt/llm_models/Llama-3.2-1B-Instruct",
    options={},
    server_url="http://0.0.0.0:8000/v1/chat/completions",
):
    """Generate text using the specified language model."""
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
    }
    payload.update(options)
    response = requests.post(server_url, headers=headers, json=payload)
    if response.status_code == 200:
        data = response.json()
        return data["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

Dataset boolq loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/boolq
Dataset cb loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/cb
Dataset copa loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/copa
Dataset multirc loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/multirc
Dataset record loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/record
Dataset rte loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/rte
Dataset wic loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/wic
Dataset wsc loaded from /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/datasets/wsc
Task: boolq
DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_

In [5]:
# BoolQ (Boolean Questions)
def format_boolq_prompt(example):
    base_instruction = (
        "Answer the following question based on the passage. Type 'True' or 'False'.\n"
    )
    return f"{base_instruction}Question: {example['question']}\nPassage: {example['passage']}\n:"


# CB (CommitmentBank)
def format_cb_prompt(example):
    base_instruction = "Answer the following question based on the Premise and Hypothesis. Type 'entailment', 'contradiction', or 'neutral'.\n"
    return f"{base_instruction}Premise: {example['premise']}\nHypothesis: {example['hypothesis']}\n Does the premise entail the hypothesis? (entailment, contradiction, or neutral):"


# COPA (Choice of Plausible Alternatives)
def format_copa_prompt(example):
    base_instruction = """Answer the following question based on the Premise and Question. Type 'choice 1' or 'choice 2'.\n If the question is "What was the cause?" select the option most likely to explain why the premise happened. If the question is "What happened as a result?" select the option most likely to occur as a result of the premise."""
    return f"{base_instruction}Premise: {example['premise']}\nQuestion: {example['question']}\nChoice 1: {example['choice1']}\nChoice 2: {example['choice2']}\n Which choice is more plausible? "


# MultiRC (Multi-Sentence Reading Comprehension)
def format_multirc_prompt(example):
    base_instruction = "Based on the provided (Paragraph, Question, Answer) pair, answer the question with 'True' or 'False'.\n"
    return f"{base_instruction}Paragraph: {example['paragraph']}\nQuestion: {example['question']}\nAnswer: {example['answer']}\nIs the answer correct? (True/False):"


# ReCoRD (Reading Comprehension with Commonsense Reasoning)
def format_record_prompt(example):
    base_instruction = "Based on the provided passage and entities, answer the question with the correct entity.\n"
    return f"{base_instruction}Passage: {example['passage']}\nQuery: {example['query']}\nEntities: {', '.join(example['entities'])}\nWhich entity best fills in the blank?"


# RTE (Recognizing Textual Entailment)
def format_rte_prompt(example):
    base_instruction = "Answer the following question based on the Premise and Hypothesis. Type 'Yes' or 'No'.\n"
    return f"{base_instruction}Premise: {example['premise']}\nHypothesis: {example['hypothesis']}\nDoes the premise entail the hypothesis? (Yes or No):"


# WiC (Word-in-Context)
def format_wic_prompt(data):
    # Extract relevant information from the input data
    word = data["word"]
    sentence1 = data["sentence1"]
    sentence2 = data["sentence2"]
    start1, end1 = data["start1"], data["end1"]
    start2, end2 = data["start2"], data["end2"]

    # Extract the highlighted word in both sentences
    sentence1_highlighted = sentence1[start1 : end1 + 1]
    sentence2_highlighted = sentence2[start2 : end2 + 1]

    # Generate the prompt text
    prompt = f"""
    Task: Determine whether the word "{word}" is used with the same meaning in both sentences below.

    Sentence 1: "{sentence1}"  
    Sentence 2: "{sentence2}"  

    The word appears in Sentence 1 as: "{sentence1_highlighted}"  
    The word appears in Sentence 2 as: "{sentence2_highlighted}"  

    Question: Is the word "{word}" used with the same meaning in both sentences?  Type (Yes, No)
    """
    return prompt


# WSC (Winograd Schema Challenge)
def format_wsc_prompt(data):
    base_instruction = "Based on the provided text and pronoun, answer the question with the correct referent.\n"
    text = data["text"]
    span1_text = data["span1_text"]
    span2_text = data["span2_text"]
    span1_index = data["span1_index"]
    span2_index = data["span2_index"]
    prompt = f"""
    {base_instruction}
    Text:  
    {text}
    
    The first mention of the entity is: "{span1_text}" at index {span1_index}
    The second mention of the entity is: "{span2_text}" at index {span2_index}
    
    Task: Identify whether the second mention of the entity refers to the same entity as the first mention.
    
    Question: Does the second mention of the entity ("{span2_text}") refer to the same entity as the first mention ("{span1_text}") in the text?  Type (Yes, No)
    """
    return prompt

In [53]:
import openai

file_path = "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/.vscode/api_key.txt"

# # Read the API key from the file
with open(file_path, "r") as file:
    openai.api_key = file.read().strip()


def call_openai_api(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()


def evaluate_prediction_with_conversion(task, example, prediction):
    # Helper functions for conversion
    def bool_to_binary(value):
        # Strip whitespace and convert to lowercase for consistency
        value = value.strip().lower()

        # Check if the string is exactly "true"
        if value == "true":
            return 1
        elif value == "false":
            return 0

        # Check if the string contains "true" but does not contain "false"
        elif "true" in value and "false" not in value:
            return 1

        # Check if the string contains "false" but does not contain "true"
        elif "false" in value and "true" not in value:
            return 0

        # If both "true" and "false" are present or neither is present, return None
        else:
            return None

    def yes_no_to_binary(value):
        # Strip whitespace and convert to lowercase for consistency
        value = value.strip().lower()

        # Check if the string is exactly "yes"
        if value == "yes":
            return 1
        elif value == "no":
            return 0

        # Check if the string contains "yes" but does not contain "no"
        elif "yes" in value and "no" not in value:
            return 1

        # Check if the string contains "no" but does not contain "yes"
        elif "no" in value and "yes" not in value:
            return 0

        # If both "yes" and "no" are present or neither is present, return None
        else:
            return None

    def entailment_to_label(value):
        # Define the mapping for entailment, contradiction, and neutral
        mapping = {"entailment": 0, "contradiction": 1, "neutral": 2}

        # Normalize the input by stripping whitespace and converting to lowercase
        value = value.strip().lower()

        # Check if the input matches exactly one of the keys in the mapping
        if value in mapping:
            return mapping[value]

        # Check if the input contains one of the keys without ambiguity
        elif (
            "entailment" in value
            and "contradiction" not in value
            and "neutral" not in value
        ):
            return mapping["entailment"]
        elif (
            "contradiction" in value
            and "entailment" not in value
            and "neutral" not in value
        ):
            return mapping["contradiction"]
        elif (
            "neutral" in value
            and "entailment" not in value
            and "contradiction" not in value
        ):
            return mapping["neutral"]

        # If the input is ambiguous or invalid, return -1
        else:
            return -1

    def choice_to_binary(value):
        # Normalize the input by stripping whitespace and converting to lowercase
        value = value.strip().lower()

        # Check if the input contains 'choice 1' and does not contain 'choice 2'
        if "choice 1" in value and "choice 2" not in value:
            return 0
        elif "choice 2" in value:
            return 1

        # If the input does not match any of the conditions, return None
        else:
            return None

    # Task-specific evaluation
    if task == "boolq":
        # Convert prediction (True/False) to binary and compare with label (0/1)
        return bool_to_binary(prediction) == int(example["label"])

    elif task == "cb":
        # Convert prediction (entailment/contradiction/neutral) to label (0/1/2)
        return entailment_to_label(prediction) == int(example["label"])

    elif task == "copa":
        # Convert prediction (choice1/choice2) to binary and compare with label (0/1)
        return choice_to_binary(prediction) == int(example["label"])

    elif task == "multirc":
        # Convert prediction (True/False) to binary and compare with label (0/1)
        return bool_to_binary(prediction) == int(example["label"])

    elif task == "record":
        # Direct comparison of prediction with the correct entity
        processed_answers = [answer.strip().lower() for answer in example["answers"]]
        return prediction.strip().lower() in processed_answers

    elif task == "rte":
        # Convert prediction (Yes/No) to binary and compare with label (1/0)
        return yes_no_to_binary(prediction) == (1 - int(example["label"]))

    elif task == "wic":
        # Convert prediction (Yes/No) to binary and compare with label (0/1)
        return yes_no_to_binary(prediction) == int(example["label"])

    elif task == "wsc":
        # Convert prediction (True/False) to binary and compare with label (0/1)
        return yes_no_to_binary(prediction) == int(example["label"])

    # Default case: unknown task
    return False


import pandas as pd
import json
from tqdm import tqdm


data_rows = []
for dataset_label in ["train", "validation", "test"]:
    for task, dataset in datasets.items():
        test_data = dataset[dataset_label][0 : len(dataset[dataset_label])]

        test_data = [
            {key: test_data[key][i] for key in test_data}
            for i in range(len(test_data[list(test_data.keys())[0]]))
        ]

        for example in tqdm(test_data, desc=f"Examples for {task}", leave=False):
            if task == "boolq":
                prompt = format_boolq_prompt(example)
            elif task == "cb":
                prompt = format_cb_prompt(example)
            elif task == "copa":
                prompt = format_copa_prompt(example)
            elif task == "multirc":
                prompt = format_multirc_prompt(example)
            elif task == "record":
                prompt = format_record_prompt(example)
            elif task == "rte":
                prompt = format_rte_prompt(example)
            elif task == "wic":
                prompt = format_wic_prompt(example)
            elif task == "wsc":
                prompt = format_wsc_prompt(example)

            data_rows.append(
                {
                    "dataset_label": dataset_label,
                    "task": task,
                    "prompt": prompt,
                    "example": example,  # Include the raw example for reference if needed
                }
            )

            # options["temperature"] = 0.1
            # prediction = generate_text_with_vllm(
            #     prompt,
            #     options=options,
            #     server_url=server_url,
            # )

            # if dataset_label != "test":
            #     label = evaluate_prediction_with_conversion(task, example, prediction)
            # else:
            #     label = None

            # result_df = pd.DataFrame(
            #     {
            #         "task": task,
            #         "example": json.dumps(example),
            #         "prompt": prompt,
            #         "prediction": prediction,
            #         "label": label,
            #     },
            #     index=[0],
            # )
            # result_df.to_json(
            #     f"/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/superGlue/results_test_{output_name}.jsonl",
            #     orient="records",
            #     lines=True,
            #     mode="a",
            # )

super_glue_df = pd.DataFrame(data_rows)

In [56]:
grouped_stats = (
    super_glue_df.groupby(["dataset_label", "task"]).size().reset_index(name="count")
)
print(grouped_stats)

Index(['dataset_label', 'task', 'prompt', 'example'], dtype='object')

In [63]:
super_glue_df.to_json(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/super_glue_data.jsonl",
    lines=True,
    orient="records",
)

## Test the superglue